### Business Case Study - Staff Planning### 

In [1]:
# Importing important Libraries 
import pandas as pd
import numpy as np
import math
from pyomo.environ import *
from __future__ import division
from pyomo.opt import SolverFactory

### Specify/Import data

In [2]:
# Reading the data from Excel workbook

StaffAV = pd.read_excel("Staffing+Data.xlsx", sheet_name= 'StaffAvailability')
Cost = pd.read_excel("Staffing+Data.xlsx", sheet_name= 'Cost')
transcost = pd.read_excel("Staffing+Data.xlsx", sheet_name= 'ServiceRate')
MgDemandDist = pd.read_excel("Staffing+Data.xlsx", sheet_name= 'DemandData')

In [3]:
MgDemandDist.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [4]:
print(MgDemandDist.set_index(['State','Month'])['Demand'])

State  Month
A      Jan      5240
       Feb      4878
       Mar      5942
       Apr      2297
       May      1992
       Jun      2275
       Jul      5334
       Aug      3371
       Sep      3759
       Oct      3529
       Nov      4284
       Dec      5183
B      Jan      4927
       Feb      2628
       Mar      2974
       Apr      2338
       May      4020
       Jun      3147
       Jul      4271
       Aug      2620
       Sep      4517
       Oct      4155
       Nov      3137
       Dec      4227
C      Jan      1162
       Feb      1967
       Mar      1898
       Apr      2261
       May      2030
       Jun      1642
       Jul      2489
       Aug      2496
       Sep       922
       Oct      2421
       Nov       963
       Dec      1998
Name: Demand, dtype: int64


In [5]:
MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

{('A', 'Jan'): 5240,
 ('A', 'Feb'): 4878,
 ('A', 'Mar'): 5942,
 ('A', 'Apr'): 2297,
 ('A', 'May'): 1992,
 ('A', 'Jun'): 2275,
 ('A', 'Jul'): 5334,
 ('A', 'Aug'): 3371,
 ('A', 'Sep'): 3759,
 ('A', 'Oct'): 3529,
 ('A', 'Nov'): 4284,
 ('A', 'Dec'): 5183,
 ('B', 'Jan'): 4927,
 ('B', 'Feb'): 2628,
 ('B', 'Mar'): 2974,
 ('B', 'Apr'): 2338,
 ('B', 'May'): 4020,
 ('B', 'Jun'): 3147,
 ('B', 'Jul'): 4271,
 ('B', 'Aug'): 2620,
 ('B', 'Sep'): 4517,
 ('B', 'Oct'): 4155,
 ('B', 'Nov'): 3137,
 ('B', 'Dec'): 4227,
 ('C', 'Jan'): 1162,
 ('C', 'Feb'): 1967,
 ('C', 'Mar'): 1898,
 ('C', 'Apr'): 2261,
 ('C', 'May'): 2030,
 ('C', 'Jun'): 1642,
 ('C', 'Jul'): 2489,
 ('C', 'Aug'): 2496,
 ('C', 'Sep'): 922,
 ('C', 'Oct'): 2421,
 ('C', 'Nov'): 963,
 ('C', 'Dec'): 1998}

In [6]:
Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [7]:
transcost

,MgAppServedPerMonth
0,40


In [8]:
StaffAV.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


### Data pre-processing 

In [9]:
# Create the required Python data structures for indexes and parameters

# Demand Data
Demand = MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

# FTE Salary from the data
FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Unit Outsourcing Cost
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Appl when employee is working full time i.e. 100% utilization
FTE_AppServeRate = transcost.iloc[0,0]

# Staff Availability for serving the Insurance Applications
StaffAv_LB = StaffAV.set_index(['State','Month'])['LB'].to_dict() 
StaffAv_UB = StaffAV.set_index(['State','Month'])['UB'].to_dict()
StaffAvPer = StaffAV.set_index(['State','Month'])['StaffAvPer'].to_dict()

# Index
BankLoc = Cost['State'].unique()
Month = Cost['Month'].unique()

In [10]:
Demand

{('A', 'Jan'): 5240,
 ('A', 'Feb'): 4878,
 ('A', 'Mar'): 5942,
 ('A', 'Apr'): 2297,
 ('A', 'May'): 1992,
 ('A', 'Jun'): 2275,
 ('A', 'Jul'): 5334,
 ('A', 'Aug'): 3371,
 ('A', 'Sep'): 3759,
 ('A', 'Oct'): 3529,
 ('A', 'Nov'): 4284,
 ('A', 'Dec'): 5183,
 ('B', 'Jan'): 4927,
 ('B', 'Feb'): 2628,
 ('B', 'Mar'): 2974,
 ('B', 'Apr'): 2338,
 ('B', 'May'): 4020,
 ('B', 'Jun'): 3147,
 ('B', 'Jul'): 4271,
 ('B', 'Aug'): 2620,
 ('B', 'Sep'): 4517,
 ('B', 'Oct'): 4155,
 ('B', 'Nov'): 3137,
 ('B', 'Dec'): 4227,
 ('C', 'Jan'): 1162,
 ('C', 'Feb'): 1967,
 ('C', 'Mar'): 1898,
 ('C', 'Apr'): 2261,
 ('C', 'May'): 2030,
 ('C', 'Jun'): 1642,
 ('C', 'Jul'): 2489,
 ('C', 'Aug'): 2496,
 ('C', 'Sep'): 922,
 ('C', 'Oct'): 2421,
 ('C', 'Nov'): 963,
 ('C', 'Dec'): 1998}

In [11]:
FTE_Salary

{('A', 'Jan'): 5000.0,
 ('A', 'Feb'): 5000.0,
 ('A', 'Mar'): 5000.0,
 ('A', 'Apr'): 5000.0,
 ('A', 'May'): 5000.0,
 ('A', 'Jun'): 5000.0,
 ('A', 'Jul'): 5000.0,
 ('A', 'Aug'): 5000.0,
 ('A', 'Sep'): 5000.0,
 ('A', 'Oct'): 5000.0,
 ('A', 'Nov'): 5000.0,
 ('A', 'Dec'): 5000.0,
 ('B', 'Jan'): 4583.333333333333,
 ('B', 'Feb'): 4583.333333333333,
 ('B', 'Mar'): 4583.333333333333,
 ('B', 'Apr'): 4583.333333333333,
 ('B', 'May'): 4583.333333333333,
 ('B', 'Jun'): 4583.333333333333,
 ('B', 'Jul'): 4583.333333333333,
 ('B', 'Aug'): 4583.333333333333,
 ('B', 'Sep'): 4583.333333333333,
 ('B', 'Oct'): 4583.333333333333,
 ('B', 'Nov'): 4583.333333333333,
 ('B', 'Dec'): 4583.333333333333,
 ('C', 'Jan'): 4416.666666666667,
 ('C', 'Feb'): 4416.666666666667,
 ('C', 'Mar'): 4416.666666666667,
 ('C', 'Apr'): 4416.666666666667,
 ('C', 'May'): 4416.666666666667,
 ('C', 'Jun'): 4416.666666666667,
 ('C', 'Jul'): 4416.666666666667,
 ('C', 'Aug'): 4416.666666666667,
 ('C', 'Sep'): 4416.666666666667,
 ('C', 'Oc

In [12]:
UnitOutSourceCost

{('A', 'Jan'): 180,
 ('A', 'Feb'): 180,
 ('A', 'Mar'): 180,
 ('A', 'Apr'): 180,
 ('A', 'May'): 180,
 ('A', 'Jun'): 180,
 ('A', 'Jul'): 180,
 ('A', 'Aug'): 180,
 ('A', 'Sep'): 180,
 ('A', 'Oct'): 180,
 ('A', 'Nov'): 180,
 ('A', 'Dec'): 180,
 ('B', 'Jan'): 150,
 ('B', 'Feb'): 150,
 ('B', 'Mar'): 150,
 ('B', 'Apr'): 150,
 ('B', 'May'): 150,
 ('B', 'Jun'): 150,
 ('B', 'Jul'): 150,
 ('B', 'Aug'): 150,
 ('B', 'Sep'): 150,
 ('B', 'Oct'): 150,
 ('B', 'Nov'): 150,
 ('B', 'Dec'): 150,
 ('C', 'Jan'): 160,
 ('C', 'Feb'): 160,
 ('C', 'Mar'): 160,
 ('C', 'Apr'): 160,
 ('C', 'May'): 160,
 ('C', 'Jun'): 160,
 ('C', 'Jul'): 160,
 ('C', 'Aug'): 160,
 ('C', 'Sep'): 160,
 ('C', 'Oct'): 160,
 ('C', 'Nov'): 160,
 ('C', 'Dec'): 160}

In [13]:
FTE_AppServeRate

40

In [14]:
StaffAV.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [15]:
StaffAv_LB

{('A', 'Jan'): 0.7,
 ('A', 'Feb'): 0.65,
 ('A', 'Mar'): 0.7,
 ('A', 'Apr'): 0.75,
 ('A', 'May'): 0.7,
 ('A', 'Jun'): 0.65,
 ('A', 'Jul'): 0.6,
 ('A', 'Aug'): 0.65,
 ('A', 'Sep'): 0.7,
 ('A', 'Oct'): 0.65,
 ('A', 'Nov'): 0.6,
 ('A', 'Dec'): 0.6,
 ('B', 'Jan'): 0.7,
 ('B', 'Feb'): 0.65,
 ('B', 'Mar'): 0.7,
 ('B', 'Apr'): 0.75,
 ('B', 'May'): 0.7,
 ('B', 'Jun'): 0.65,
 ('B', 'Jul'): 0.6,
 ('B', 'Aug'): 0.65,
 ('B', 'Sep'): 0.7,
 ('B', 'Oct'): 0.65,
 ('B', 'Nov'): 0.6,
 ('B', 'Dec'): 0.6,
 ('C', 'Jan'): 0.7,
 ('C', 'Feb'): 0.65,
 ('C', 'Mar'): 0.7,
 ('C', 'Apr'): 0.75,
 ('C', 'May'): 0.7,
 ('C', 'Jun'): 0.65,
 ('C', 'Jul'): 0.6,
 ('C', 'Aug'): 0.65,
 ('C', 'Sep'): 0.7,
 ('C', 'Oct'): 0.65,
 ('C', 'Nov'): 0.6,
 ('C', 'Dec'): 0.6}

In [16]:
StaffAv_UB

{('A', 'Jan'): 0.9,
 ('A', 'Feb'): 0.85,
 ('A', 'Mar'): 0.8,
 ('A', 'Apr'): 0.85,
 ('A', 'May'): 0.85,
 ('A', 'Jun'): 0.8,
 ('A', 'Jul'): 0.75,
 ('A', 'Aug'): 0.85,
 ('A', 'Sep'): 0.9,
 ('A', 'Oct'): 0.8,
 ('A', 'Nov'): 0.75,
 ('A', 'Dec'): 0.7,
 ('B', 'Jan'): 0.9,
 ('B', 'Feb'): 0.85,
 ('B', 'Mar'): 0.8,
 ('B', 'Apr'): 0.85,
 ('B', 'May'): 0.85,
 ('B', 'Jun'): 0.8,
 ('B', 'Jul'): 0.75,
 ('B', 'Aug'): 0.85,
 ('B', 'Sep'): 0.9,
 ('B', 'Oct'): 0.8,
 ('B', 'Nov'): 0.75,
 ('B', 'Dec'): 0.7,
 ('C', 'Jan'): 0.9,
 ('C', 'Feb'): 0.85,
 ('C', 'Mar'): 0.8,
 ('C', 'Apr'): 0.85,
 ('C', 'May'): 0.85,
 ('C', 'Jun'): 0.8,
 ('C', 'Jul'): 0.75,
 ('C', 'Aug'): 0.85,
 ('C', 'Sep'): 0.9,
 ('C', 'Oct'): 0.8,
 ('C', 'Nov'): 0.75,
 ('C', 'Dec'): 0.7}

In [17]:
StaffAvPer

{('A', 'Jan'): 0.81,
 ('A', 'Feb'): 0.76,
 ('A', 'Mar'): 0.75,
 ('A', 'Apr'): 0.8,
 ('A', 'May'): 0.78,
 ('A', 'Jun'): 0.73,
 ('A', 'Jul'): 0.68,
 ('A', 'Aug'): 0.76,
 ('A', 'Sep'): 0.81,
 ('A', 'Oct'): 0.73,
 ('A', 'Nov'): 0.68,
 ('A', 'Dec'): 0.65,
 ('B', 'Jan'): 0.81,
 ('B', 'Feb'): 0.76,
 ('B', 'Mar'): 0.75,
 ('B', 'Apr'): 0.8,
 ('B', 'May'): 0.78,
 ('B', 'Jun'): 0.73,
 ('B', 'Jul'): 0.68,
 ('B', 'Aug'): 0.76,
 ('B', 'Sep'): 0.81,
 ('B', 'Oct'): 0.73,
 ('B', 'Nov'): 0.68,
 ('B', 'Dec'): 0.65,
 ('C', 'Jan'): 0.81,
 ('C', 'Feb'): 0.76,
 ('C', 'Mar'): 0.75,
 ('C', 'Apr'): 0.8,
 ('C', 'May'): 0.78,
 ('C', 'Jun'): 0.73,
 ('C', 'Jul'): 0.68,
 ('C', 'Aug'): 0.76,
 ('C', 'Sep'): 0.81,
 ('C', 'Oct'): 0.73,
 ('C', 'Nov'): 0.68,
 ('C', 'Dec'): 0.65}

In [18]:
BankLoc

array(['A', 'B', 'C'], dtype=object)

In [19]:
Month

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

### Question 1
The company wants to know the optimised staffing recommendations for the business case described. Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint.

In [20]:
BankLoc.tolist()

['A', 'B', 'C']

In [21]:
# Creating an object of Concrete Model
model = ConcreteModel()

# defining  the Sets
# i Location of States (A, B, C)
# j Months (Jan, Feb, Mar, ......, Dec)

model.i = Set(initialize=BankLoc.tolist(), doc='States')
model.j = Set(initialize=Month.tolist(), doc='Months')

# Defining Parameters for Demand, FTE Salaries, OutsourcingCost, Average Staff Availability
model.d = Param(model.i, model.j, initialize=Demand, doc='Demand')
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc='FTE_Salary')
model.oc = Param(model.i, model.j, initialize=UnitOutSourceCost, doc='OutSourceCost')
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc='StaffAvPercent')

# Scalar Application Service Rate when working 100%
model.r =Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

In [22]:
# Parameter for the number of applications that can be processed for the given staff availability

def c_FTEAPP(model, i, j):
    return model.r * model.sa[i,j] 

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc='NO OF FTE APP')

In [23]:
# Parameters for the restriction on Outsourced Applications for State A

def Reg_A(model, i, j):
    return 0.30 * model.d[i,j] # 30% of Demand

model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc='RegRest_A')

In [24]:
# Parameters for the restriction on Outsourced Applications for State B

def Reg_B(model, i, j):
    return 0.40 * model.d[i,j] # 40% of Demand

model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc='RegRest_B')


In [25]:
model.i

In [26]:
# Define Decision Variables

model.x = Var(model.i, model.j, domain=NonNegativeReals, doc='No of FTE')
model.y = Var(model.i, model.j, domain=NonNegativeIntegers, doc='No of Outsourced App')

In [27]:
# Demand Constraint

model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr= model.x[i,j] * model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [28]:
# Regulatory Constraint

model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i == 'A':
            model.regulatoryCons.add(expr= model.y[i,j] <= model.Reg_A['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr= model.y[i,j] <= model.Reg_B['B',j])

In [29]:
# Define Objective Function

def objective_rule(model):
    return sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) +\
           sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule= objective_rule, sense= minimize, doc='Define objective function')

In [30]:
# SOLVE

result_cost = SolverFactory('glpk').solve(model)                        
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.13691949844360352
# ----------------------------------------------------------
#   Solution Information
# ------------

In [31]:
model.pprint()

13 Set Declarations
    FTEAPPNO_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Reg_A_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), (

In [32]:
# Print the value of the objective function
optimized_cost = round(model.objective()/1000000,2)
optimized_cost

17.96

-  Looks like the company has to spend around 18m $ in total for the application approval process.

### Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

## Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [33]:
#Creating an empty list
Output_actual = []


for i in model.i:
    for j in model.j:
        
        #cost for the application approval process
        no_of_fte=model.x[i,j].value
        demand = model.d[i, j]
        no_of_outsource_app = model.y[i,j].value
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        cost = model.x[i,j].value*FTE_Salary[i,j] + model.y[i,j].value*UnitOutSourceCost[i,j]
        percent_outsourced_app = round((no_of_outsource_app/demand)*100,2)
        avg_cost_per_app = round(cost/demand,2)
        
        Output_actual.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
        
print(Output_actual)

[['A', 'Jan', 161.728395061728, 5240, 0.0, 5239.999999999987, 808641.97530864, 0.0, 154.32], ['A', 'Feb', 160.460526315789, 4878, 0.0, 4877.999999999985, 802302.631578945, 0.0, 164.47], ['A', 'Mar', 198.066666666667, 5942, 0.0, 5942.000000000011, 990333.333333335, 0.0, 166.67], ['A', 'Apr', 71.78125, 2297, 0.0, 2297.0, 358906.25, 0.0, 156.25], ['A', 'May', 63.8461538461538, 1992, 0.0, 1991.9999999999986, 319230.76923076896, 0.0, 160.26], ['A', 'Jun', 77.9109589041096, 2275, 0.0, 2275.0000000000005, 389554.794520548, 0.0, 171.23], ['A', 'Jul', 137.279411764706, 5334, 1600.0, 3734.000000000003, 974397.05882353, 30.0, 182.68], ['A', 'Aug', 110.888157894737, 3371, 0.0, 3371.0000000000055, 554440.7894736851, 0.0, 164.47], ['A', 'Sep', 116.018518518519, 3759, 0.0, 3759.000000000016, 580092.592592595, 0.0, 154.32], ['A', 'Oct', 120.856164383562, 3529, 0.0, 3529.0000000000105, 604280.82191781, 0.0, 171.23], ['A', 'Nov', 110.257352941176, 4284, 1285.0, 2998.9999999999873, 782586.76470588, 30.0,

In [35]:
Output_actual = pd.DataFrame(Output_actual, columns = ['State', 'Month', 'No of FTE',"Demand",'No of outsource app',
                                                       "No of FTE processed App" , 'Estimated Cost', 'percent outsourced app', 
                                                       'average cost per application'])

Output_actual[['Estimated Cost in mn $']] = Output_actual[['Estimated Cost']]/1000000
Output_actual['Demand'] = Output_actual['Demand'].astype(int)
Output_actual['No of outsource app'] = Output_actual['No of outsource app'].astype(int)
Output_actual['No of FTE processed App'] = Output_actual['No of FTE processed App'].astype(int)
Output_actual = Output_actual.round({"No of FTE":1, "No of outsource app":0, "Estimated Cost in mn $":1})


In [36]:
Output_actual.head()

,State,Month,No of FTE,Demand,No of outsource app,No of FTE processed App,Estimated Cost,percent outsourced app,average cost per application,Estimated Cost in mn $
0,A,Jan,161.7,5240,0,5239,808641.975309,0.0,154.32,0.8
1,A,Feb,160.5,4878,0,4877,802302.631579,0.0,164.47,0.8
2,A,Mar,198.1,5942,0,5942,990333.333333,0.0,166.67,1.0
3,A,Apr,71.8,2297,0,2297,358906.250000,0.0,156.25,0.4
4,A,May,63.8,1992,0,1991,319230.769231,0.0,160.26,0.3


In [37]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_actual'
Output_actual.to_excel(writer, sheet_name='Output_actual')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


### Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [38]:
# Creating a model instance
WC_model = ConcreteModel()

In [39]:
# Define Pyomo sets and Parameters
WC_model.i = Set(initialize= BankLoc.tolist(),doc='States')
WC_model.j = Set(initialize= Month.tolist(),doc='Months')
WC_model.demand = Param(WC_model.i,WC_model.j,initialize = Demand,doc='Demand')
WC_model.sa = Param(WC_model.i,WC_model.j,initialize = StaffAv_LB,doc = 'StaffLowerBoundPercent')

In [40]:
# Decision variables
WC_model.x = Var(WC_model.i,WC_model.j,doc='No of FTE',domain = NonNegativeReals)
WC_model.y = Var(WC_model.i,WC_model.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [41]:
# Demand Constraint

def total_demand(m, i, j):
    return (m.x[i, j] * 40 * m.sa[i, j] + m.y[i, j] == m.demand[i, j])


WC_model.total_demand = Constraint(WC_model.i, WC_model.j, rule=total_demand)

# Outsource Demand Constraints
WC_model.outsource_demand = ConstraintList()
for j in WC_model.j:
    for i in WC_model.i:
        if i == 'A':
            WC_model.outsource_demand.add(expr = WC_model.y[i,j] <= 0.3* WC_model.demand[i,j])
        elif i == 'B':
            WC_model.outsource_demand.add(expr = WC_model.y[i,j] <= 0.4* WC_model.demand[i,j])

In [42]:
# Objective function
WC_model.value = Objective(expr = sum(sum(WC_model.x[i,j]*FTE_Salary[i,j]+ WC_model.y[i,j]
                                        *UnitOutSourceCost[i,j] for i in WC_model.i) 
                                    for j in WC_model.j),sense= minimize)

In [43]:
# Invoking the solver
result = SolverFactory('glpk').solve(WC_model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08495068550109863
# ----------------------------------------------------------
#   Solution Information
# ------------

In [44]:
# Print the value of the objective function
optimized_cost_worst = round(WC_model.value()/1000000,2)

### The company need to spend around $19.6m in total for the application approval process


In [45]:
WC_model.pprint()

8 Set Declarations
    demand_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    i : States
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    j : Months
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {'Jan', 'Feb', 'Mar', 'Apr',

In [46]:
# Creating dataframe for the results
Output_worst = []

for i in WC_model.i:
    for j in WC_model.j:
       
        no_of_fte = WC_model.x[i, j].value
        demand = WC_model.demand[i, j]
        no_of_outsource_app = WC_model.y[i, j].value
        no_of_FTE_processed_app = WC_model.x[i,j].value*40*WC_model.sa[i,j]
        cost = WC_model.x[i, j].value*FTE_Salary[i, j] + WC_model.y[i, j].value*UnitOutSourceCost[i, j]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        avg_cost_per_app = round(cost/demand,1)

        Output_worst.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, 
                             percent_outsourced_app,avg_cost_per_app])
print(Output_worst)        

[['A', 'Jan', 187.142857142857, 5240, 0.0, 5239.999999999995, 935714.285714285, 0.0, 178.6], ['A', 'Feb', 131.346153846154, 4878, 1463.0, 3415.0000000000045, 920070.7692307701, 30.0, 188.6], ['A', 'Mar', 212.214285714286, 5942, 0.0, 5942.000000000008, 1061071.42857143, 0.0, 178.6], ['A', 'Apr', 76.5666666666667, 2297, 0.0, 2297.0000000000014, 382833.33333333355, 0.0, 166.7], ['A', 'May', 71.1428571428571, 1992, 0.0, 1991.9999999999986, 355714.2857142855, 0.0, 178.6], ['A', 'Jun', 61.2692307692308, 2275, 682.0, 1593.0000000000011, 429106.153846154, 30.0, 188.6], ['A', 'Jul', 155.583333333333, 5334, 1600.0, 3733.999999999992, 1065916.666666665, 30.0, 199.8], ['A', 'Aug', 90.7692307692308, 3371, 1011.0, 2360.000000000001, 635826.153846154, 30.0, 188.6], ['A', 'Sep', 134.25, 3759, 0.0, 3758.9999999999995, 671250.0, 0.0, 178.6], ['A', 'Oct', 95.0384615384615, 3529, 1058.0, 2470.999999999999, 665632.3076923075, 30.0, 188.6], ['A', 'Nov', 124.958333333333, 4284, 1285.0, 2998.9999999999923, 85

In [47]:
Output_worst = pd.DataFrame(Output_worst, columns = ['State', 'Month', 'No of FTE worst',"Demand",
                                                     'No of outsource app worst',"No of FTE processed App worst" ,
                                                     'Estimated Cost worst', 'percent outsourced app worst', 
                                                     'average cost per application worst'])

Output_worst['Estimated Cost in mn $ worst'] = Output_worst[['Estimated Cost worst']]/1000000
Output_worst['Demand'] = Output_worst['Demand'].astype(int)
Output_worst['No of outsource app worst'] = Output_worst['No of outsource app worst'].astype(int)
Output_worst['No of FTE processed App worst'] = Output_worst['No of FTE processed App worst'].astype(int)
Output_worst = Output_worst.round({"No of FTE worst":1, "No of outsource app worst":0, "Estimated Cost in mn $ worst":1})

print("\n\n" ,Output_worst)



    State Month  No of FTE worst  Demand  No of outsource app worst  \
0      A   Jan            187.1    5240                          0   
1      A   Feb            131.3    4878                       1463   
2      A   Mar            212.2    5942                          0   
3      A   Apr             76.6    2297                          0   
4      A   May             71.1    1992                          0   
5      A   Jun             61.3    2275                        682   
6      A   Jul            155.6    5334                       1600   
7      A   Aug             90.8    3371                       1011   
8      A   Sep            134.2    3759                          0   
9      A   Oct             95.0    3529                       1058   
10     A   Nov            125.0    4284                       1285   
11     A   Dec            151.2    5183                       1554   
12     B   Jan            105.6    4927                       1970   
13     B   Feb   

In [48]:
total_outsourcing =  sum(Output_worst['No of outsource app worst'])
total_cost = sum(Output_worst['Estimated Cost worst'])
optimal_staff_members = round(Output_worst['No of FTE worst'].mean(),1)

total_demand =  sum(Output_worst['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)

print("Total Outsourced Application: ", total_outsourcing)
print("Total Demand Application: ", total_demand)
print("Optimal Staff Members : ", optimal_staff_members)

Total Outsourced Application:  39808
Total Demand Application:  113294
Optimal Staff Members :  76.7


### 3.1.2 Percentage of outsourced applications 

In [49]:
# write your code here
print("Total Outsourced Application % is : ", percent_outsourced_app_total)


Total Outsourced Application % is :  35.1


### 3.1.3 Average cost per application

In [50]:

print("Average Cost per Application: ", average_cost_per_appln)

Average Cost per Application:  173.0


In [51]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_worst'
Output_worst.to_excel(writer, sheet_name='Output_worst')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


### 3.2  Best Case Analysis 

In [52]:
# Creating a model instance
BC_model = ConcreteModel()

In [53]:
# Define Pyomo sets and Parameters
BC_model.i = Set(initialize=BankLoc.tolist(),doc='States')
BC_model.j = Set(initialize=Month.tolist(),doc='Months')


BC_model.demand = Param(BC_model.i,BC_model.j,initialize = Demand,doc='Demand')
BC_model.sa = Param(BC_model.i,BC_model.j,initialize = StaffAv_UB,doc = 'StaffUpperBoundPercent')

In [54]:
# Decision variables
BC_model.x = Var(BC_model.i,BC_model.j,doc='No of FTE',domain = NonNegativeReals)
BC_model.y = Var(BC_model.i,BC_model.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [55]:
# Demand Constraint

def total_demand(m, i, j):
    return (m.x[i, j] * 40 * m.sa[i, j] + m.y[i, j] == m.demand[i, j])


BC_model.total_demand = Constraint(BC_model.i, BC_model.j, rule=total_demand)

# Outsource Demand Constraints
BC_model.outsource_demand = ConstraintList()
for j in BC_model.j:
    for i in BC_model.i:
        if i == 'A':
            BC_model.outsource_demand.add(expr = BC_model.y[i,j] <= 0.3*BC_model.demand[i,j])
        elif i == 'B':
            BC_model.outsource_demand.add(expr = BC_model.y[i,j] <= 0.4*BC_model.demand[i,j])

In [56]:
# Objective function
BC_model.value = Objective(expr = sum(sum(BC_model.x[i,j]*FTE_Salary[i,j]+
                                        BC_model.y[i,j]*UnitOutSourceCost[i,j] 
                                        for i in BC_model.i) for j in BC_model.j),
                         sense= minimize)


In [57]:
# Invoking the solver
result = SolverFactory('glpk').solve(BC_model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08794927597045898
# ----------------------------------------------------------
#   Solution Information
# ------------

In [58]:
# Print the value of the objective function
optimized_cost_best = round(BC_model.value()/1000000,2)

### **`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [59]:
BC_model.pprint

<bound method Component.pprint of <pyomo.core.base.PyomoModel.ConcreteModel object at 0x0000026B4C0CFE80>>

In [60]:
# Creating dataframe for the results
Output_best = []

for i in BC_model.i:
    for j in BC_model.j:
        # cost for the application approval process
        no_of_fte = BC_model.x[i, j].value
        demand = BC_model.demand[i, j]
        no_of_outsource_app = BC_model.y[i, j].value
        no_of_FTE_processed_app = BC_model.x[i,j].value*40*BC_model.sa[i,j]
        cost = BC_model.x[i, j].value*FTE_Salary[i, j] + BC_model.y[i, j].value*UnitOutSourceCost[i, j]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        cost_per_app = round(cost/demand,1)

        Output_best.append([i, j, no_of_fte,demand, 
                            no_of_outsource_app,no_of_FTE_processed_app, 
                            cost, percent_outsourced_app,cost_per_app])
print(Output_best)

[['A', 'Jan', 145.555555555556, 5240, 0.0, 5240.000000000016, 727777.77777778, 0.0, 138.9], ['A', 'Feb', 143.470588235294, 4878, 0.0, 4877.999999999996, 717352.94117647, 0.0, 147.1], ['A', 'Mar', 185.6875, 5942, 0.0, 5942.0, 928437.5, 0.0, 156.2], ['A', 'Apr', 67.5588235294118, 2297, 0.0, 2297.0000000000014, 337794.11764705897, 0.0, 147.1], ['A', 'May', 58.5882352941176, 1992, 0.0, 1991.9999999999984, 292941.176470588, 0.0, 147.1], ['A', 'Jun', 71.09375, 2275, 0.0, 2275.0, 355468.75, 0.0, 156.2], ['A', 'Jul', 177.8, 5334, 0.0, 5334.0, 889000.0, 0.0, 166.7], ['A', 'Aug', 99.1470588235294, 3371, 0.0, 3370.9999999999995, 495735.29411764705, 0.0, 147.1], ['A', 'Sep', 104.416666666667, 3759, 0.0, 3759.000000000012, 522083.333333335, 0.0, 138.9], ['A', 'Oct', 110.28125, 3529, 0.0, 3529.0, 551406.25, 0.0, 156.2], ['A', 'Nov', 142.8, 4284, 0.0, 4284.0, 714000.0, 0.0, 166.7], ['A', 'Dec', 185.107142857143, 5183, 0.0, 5183.000000000004, 925535.714285715, 0.0, 178.6], ['B', 'Jan', 136.86111111111

In [61]:
Output_best = pd.DataFrame(Output_best, columns = ['State', 'Month', 'No of FTE best',
                                                   "Demand",'No of outsource app best',
                                                   "No of FTE processed App best" , 
                                                   'Estimated Cost best', 'percent outsourced app best', 
                                                   'cost per application best'])

Output_best[['Estimated Cost in mn $ best']] = Output_best[['Estimated Cost best']]/1000000
Output_best['Demand'] = Output_best['Demand'].astype(int)
Output_best['No of outsource app best'] = Output_best['No of outsource app best'].astype(int)
Output_best['No of FTE processed App best'] = Output_best['No of FTE processed App best'].astype(int)
Output_best = Output_best.round({"No of FTE best":1, "No of outsource app best":0, "Estimated Cost in mn $ best":1})



In [62]:
print("\n\n" ,Output_best)



    State Month  No of FTE best  Demand  No of outsource app best  \
0      A   Jan           145.6    5240                         0   
1      A   Feb           143.5    4878                         0   
2      A   Mar           185.7    5942                         0   
3      A   Apr            67.6    2297                         0   
4      A   May            58.6    1992                         0   
5      A   Jun            71.1    2275                         0   
6      A   Jul           177.8    5334                         0   
7      A   Aug            99.1    3371                         0   
8      A   Sep           104.4    3759                         0   
9      A   Oct           110.3    3529                         0   
10     A   Nov           142.8    4284                         0   
11     A   Dec           185.1    5183                         0   
12     B   Jan           136.9    4927                         0   
13     B   Feb            77.3    2628       

In [63]:
total_outsourcing =  sum(Output_best['No of outsource app best'])
total_cost = sum(Output_best['Estimated Cost best'])
optimal_staff_members = round(Output_best['No of FTE best'].mean(),2)

total_demand =  sum(Output_best['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)


### 3.2.2 Percentage of outsourced applications
`

In [64]:
print(percent_outsourced_app_total)


4.1


In [65]:
#### 3.2.3 Average cost per application
# write your code here
print("Average Cost per Application: ", average_cost_per_appln)

Average Cost per Application:  145.88


In [66]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_best'
Output_best.to_excel(writer, sheet_name='Output_best')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


### Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case).

### **Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.